# Terminating Conversations Between Agents

-이번 챕터에서는 AutoGen 에이전트 간의 **대화를 어떻게 종료할 수 있는지**에 대해 살펴본다. 대화 종료가 필요한 이유는, 복잡하고 자율적인 워크플로우에서 작업을 언제 끝내야 할지 아는 것이 매우 중요하기 때문이다.

예를 들어, 작업이 완료되었거나, 리소스를 너무 많이 소모했거나, 또는 다른 전략(user intervention, 사용자 개입 등)으로 전환이 필요할 때, 워크플로우를 종료하거나 흐름을 제어할 필요가 있다. 그래서 AutoGen은 여러 가지 대화 종료 메커니즘을 기본적으로 지원한다.

**AutoGen에서 대화 종료를 제어하는 방법** 으로는 현재 AutoGen에서 `두 가지 주요 방식`으로 **에이전트 간 대화 종료**를 제어할 수 있다.

1. `initiate_chat()` 에서 종료 조건 지정하기
-> 대화를 시작할 때, 대화 종료 시점을 제어하는 파라미터(`max_turns` 등)을 설정할 수 있다.

2. 에이전트 자체에 종료 조건을 설정하기
-> 에이전트를 정의할 때, **특정 조건이 충족되면 대화를 종료**하도록 설정할 수 있다.
이 조건은 사용자가 원하는 방식으로 **구성(configurable) 가능**하다. 

## Parameters in `initiate_chat`

이전 챕터에서, 이 기능을 한번 언급한 바 있는데 바로 `max_turns` 파라미터를 사용해서 대화 횟수를 제한했던 것이다. 예를 들어 `max_turns` 값을 3으로 늘리면, 대화가 종료되기까지 더 많은 라운드가 오가는 것을 볼 수 있다.

즉, **`max_turns`는 대화가 몇 번의 주고받음 후에 자동 종료될지를 결정하는 중요한 파라미터**이다. 이를 통해 **자율적인 흐름을 제어**할 수 있는 기본적인 수단을 제공하는 것이다. 

In [1]:
import os
from autogen import ConversableAgent

In [2]:
from config import settings

api_key = settings.openai_api_key.get_secret_value()

In [5]:
llm_config_1 = {
    "config_list" :
        [
            {
                "model" : "gpt-4o-mini",
                "api_key" : api_key,
            }
        ],
    "temperature" : 0.9,
}

llm_config_2 = {
    "config_list":
        [
            {
                "model" : "gpt-4o-mini",
                "api_key" : api_key,
            }
        ],
    "temperature" : 0.7,
}

In [6]:
yuRi = ConversableAgent(
    "유리",
    system_message= "Your name is 유리 and you are a part of a duo of comedians.",
    llm_config= llm_config_1,
    human_input_mode="NEVER",
)

miSung = ConversableAgent(
    "미성",
    system_message = "Your name is 미성 and you are a part of a duo of comedians",
    llm_config = llm_config_2,
    human_input_mode="NEVER",
)

In [7]:
result = miSung.initiate_chat(yuRi, message="유리, 농담 하나만 해줘", max_turns=2)

미성 (to 유리):

유리, 농담 하나만 해줘

--------------------------------------------------------------------------------
유리 (to 미성):

물론이지! 여기 농담 하나 있어:

왜 컴퓨터는 바다에 가고 싶어할까?

왜냐하면 너무 많은 "데이터"를 가지고 있기 때문이야! 😄

--------------------------------------------------------------------------------
미성 (to 유리):

미성: 오, 좋네! 그럼 나도 하나 해볼게! 

왜 해적들은 좋은 코미디언이 될 수 없을까? 

왜냐하면 그들은 항상 "조크"를 놓치거든! 🏴‍☠️😂

--------------------------------------------------------------------------------
유리 (to 미성):

유리: 아하, 정말 재밌네! 해적들이 농담을 놓칠 줄은 몰랐어! 😄 그럼 나도 하나 더 할게! 

왜 수학책이 슬퍼했을까?

많은 문제를 안고 있어서야! 📚😂

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (2597516c-481b-4c8c-967b-3bd9027e330a): Maximum turns (2) reached


In [8]:
result

ChatResult(chat_id=None, chat_history=[{'content': '유리, 농담 하나만 해줘', 'role': 'assistant', 'name': '미성'}, {'content': '물론이지! 여기 농담 하나 있어:\n\n왜 컴퓨터는 바다에 가고 싶어할까?\n\n왜냐하면 너무 많은 "데이터"를 가지고 있기 때문이야! 😄', 'role': 'user', 'name': '유리'}, {'content': '미성: 오, 좋네! 그럼 나도 하나 해볼게! \n\n왜 해적들은 좋은 코미디언이 될 수 없을까? \n\n왜냐하면 그들은 항상 "조크"를 놓치거든! 🏴\u200d☠️😂', 'role': 'assistant', 'name': '미성'}, {'content': '유리: 아하, 정말 재밌네! 해적들이 농담을 놓칠 줄은 몰랐어! 😄 그럼 나도 하나 더 할게! \n\n왜 수학책이 슬퍼했을까?\n\n많은 문제를 안고 있어서야! 📚😂', 'role': 'user', 'name': '유리'}], summary='유리: 아하, 정말 재밌네! 해적들이 농담을 놓칠 줄은 몰랐어! 😄 그럼 나도 하나 더 할게! \n\n왜 수학책이 슬퍼했을까?\n\n많은 문제를 안고 있어서야! 📚😂', cost={'usage_including_cached_inference': {'total_cost': 0.0001416, 'gpt-4o-mini-2024-07-18': {'cost': 0.0001416, 'prompt_tokens': 280, 'completion_tokens': 166, 'total_tokens': 446}}, 'usage_excluding_cached_inference': {'total_cost': 0.0001416, 'gpt-4o-mini-2024-07-18': {'cost': 0.0001416, 'prompt_tokens': 280, 'completion_tokens': 166, 'total_tokens': 446}}}, human_input=[])

In [9]:
result = miSung.initiate_chat(yuRi, message="유리, 재밌는 농담 해줘.", max_turns=3)

미성 (to 유리):

유리, 재밌는 농담 해줘.

--------------------------------------------------------------------------------
유리 (to 미성):

물론이지! 그럼 이 농담은 어때?

왜 컴퓨터는 항상 춤을 출 수 있을까?

왜냐하면, 그들은 "비트"가 있기 때문이야! 😂 

어때, 마음에 들었어?

--------------------------------------------------------------------------------
미성 (to 유리):

미성: 오, 그거 정말 재밌다! 나도 하나 해볼게.

왜 수학책은 항상 슬퍼할까?

왜냐하면, 너무 많은 문제를 안고 있으니까! 😂

어때, 유리? 우리 수학 책 한 권 사서 같이 읽어볼까?

--------------------------------------------------------------------------------
유리 (to 미성):

유리: 하하, 그거 진짜 재미있네! 수학책이 슬픈 이유는 정말 공감돼! 😂

그럼 우리 수학 책 대신 '웃음책' 하나 사서 같이 읽어보는 게 어때? 더 많은 문제를 웃음으로 풀 수 있을 거야!

--------------------------------------------------------------------------------
미성 (to 유리):

미성: 아하, '웃음책' 좋다! 그 안에는 "웃음 문제"들이 가득할 거야! 😂

그럼 그 책 첫 페이지에는 이런 질문이 있을 거야: "왜 웃음은 더운 날에 더 잘 퍼질까?" 

답은 "웃음은 시원한 바람과 같아서!" 

우리의 유머를 바람처럼 퍼뜨려 보자! 🌬️😄

--------------------------------------------------------------------------------
유리 (to 미성):

유리: 와, 그거 너무 웃겨! 😂 바람처럼 퍼지는 유머라니, 정말 멋진 비유야! 

그럼

In [10]:
result

ChatResult(chat_id=None, chat_history=[{'content': '유리, 재밌는 농담 해줘.', 'role': 'assistant', 'name': '미성'}, {'content': '물론이지! 그럼 이 농담은 어때?\n\n왜 컴퓨터는 항상 춤을 출 수 있을까?\n\n왜냐하면, 그들은 "비트"가 있기 때문이야! 😂 \n\n어때, 마음에 들었어?', 'role': 'user', 'name': '유리'}, {'content': '미성: 오, 그거 정말 재밌다! 나도 하나 해볼게.\n\n왜 수학책은 항상 슬퍼할까?\n\n왜냐하면, 너무 많은 문제를 안고 있으니까! 😂\n\n어때, 유리? 우리 수학 책 한 권 사서 같이 읽어볼까?', 'role': 'assistant', 'name': '미성'}, {'content': "유리: 하하, 그거 진짜 재미있네! 수학책이 슬픈 이유는 정말 공감돼! 😂\n\n그럼 우리 수학 책 대신 '웃음책' 하나 사서 같이 읽어보는 게 어때? 더 많은 문제를 웃음으로 풀 수 있을 거야!", 'role': 'user', 'name': '유리'}, {'content': '미성: 아하, \'웃음책\' 좋다! 그 안에는 "웃음 문제"들이 가득할 거야! 😂\n\n그럼 그 책 첫 페이지에는 이런 질문이 있을 거야: "왜 웃음은 더운 날에 더 잘 퍼질까?" \n\n답은 "웃음은 시원한 바람과 같아서!" \n\n우리의 유머를 바람처럼 퍼뜨려 보자! 🌬️😄', 'role': 'assistant', 'name': '미성'}, {'content': '유리: 와, 그거 너무 웃겨! 😂 바람처럼 퍼지는 유머라니, 정말 멋진 비유야! \n\n그럼 다음 페이지에는 이런 질문도 있을 거야: “왜 기분이 좋은 날은 항상 햇살이 비칠까?” \n\n답은 “긍정적인 에너지는 햇살처럼 밝기 때문이지!” ☀️ \n\n우리의 유머가 세상에 긍정적인 에너지를 불어넣자! 🌈✨', 'role': 'user', 'name': '유리'}], summary='유리:


## Agent-triggered termination

위의 방법 외에 에이전트의 파라미터를 설정해서 대화를 종료할 수도 있다.
현재 설정할 수 있는 주요 파라미터는 다음 두 가지이다.

1. **`max_consecutive_auto_reply`**
- 이 조건은 동일한 상대방에게 자동 응답을 연속으로 일정 횟수 이상 보낼 경우 대화를 종료하게 만든다.
- `conversableAgent`  클래스를 정의할 때 `max_consecutive_auto_reply` 인자를 사용해 이 값을 설정할 수 있다.
- 이 기능은 에이전트가 같은 상대에게 연속으로 몇 번 응답했는지 세는 카운터를 내부적으로 유지하며 동작한다.
- 단, 이 카운터는 **사용자의 개입(human input)**이 있으면 초기화 될 수 있다. 
-> 이에 대해서는 다음 챕터에서 더 자세히 설명한다. 

2. **`Is_termination_msg`**
- 이 조건은 받은 메시지가 특정 조건을 만족하면 대화를 종료시킨다.
예를들어 메시지에 "TERMINATE" 라는 단어가 포함되는 경우에 종료 시킬 수 있다.
- `ConversableAgent` 클래스의 생성자에서 `is_termination_msg` 인자를 설정하여 종료 조건을 **사용자 정의 함수**로 지정할 수 있다.

이 두 가지를 적절히 활용하면, 에이전트 대화를 더 유연하고 안전하게 제어할 수 있다.

### Using `max_consecutive_auto_reply`

`max_consecutive_auto_reply` 값을 1로 설정해서, miSung 에이전트가 한 번만 응답하고 대화를 종료하게 해보자. 즉, 이 설정은 동일한 상대에게 연속으로 몇 번까지 자동 응답할 수 있는지를 제한해서 불필요한 반복 대화를 방지할 수 있게 도와준다. 

In [11]:
miSung = ConversableAgent(
    "미성",
    system_message= "Your name is 미성 and your are a part of a duo of comedians.",
    llm_config = llm_config_2,
    human_input_mode= "NEVER",
    max_consecutive_auto_reply=1, # Limit the number of consecutive auto-replies.
)

In [12]:
result = miSung.initiate_chat(yuRi, message="유리. 재밌는 농담 하나만 해줘.")

미성 (to 유리):

유리. 재밌는 농담 하나만 해줘.

--------------------------------------------------------------------------------
유리 (to 미성):

물론이지! 그럼 이 농담 어때?

왜 바나나가 병원에 갔을까?

답: 껍질을 벗어야 했거든! 🍌😂 

어때, 웃겼어?

--------------------------------------------------------------------------------
미성 (to 유리):

아하, 바나나 농담 좋다! 그럼 나도 하나 해볼게.

토끼가 학교에 왜 안 갔을까?

답: 너무 많은 숙제가 있어서 '토끼'를 두고 갔거든! 🐰📚 

어때? 웃겼어?

--------------------------------------------------------------------------------
유리 (to 미성):

유리: 아하, 그거 진짜 재밌다! '토끼'를 두고 갔다는 아이디어가 기발해! 😂

그럼, 우리 농담 대결 한 번 더 해볼까? 내가 하나 해볼게!

왜 수박은 수영을 못 할까?

답: 물에 뜨는 게 힘들어서! 🍉🏊‍♂️

너무 웃기지?

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (ee802a00-b7f4-492c-bb37-461ce3e44f76): Maximum number of consecutive auto-replies reached


In [13]:
result

ChatResult(chat_id=None, chat_history=[{'content': '유리. 재밌는 농담 하나만 해줘.', 'role': 'assistant', 'name': '미성'}, {'content': '물론이지! 그럼 이 농담 어때?\n\n왜 바나나가 병원에 갔을까?\n\n답: 껍질을 벗어야 했거든! 🍌😂 \n\n어때, 웃겼어?', 'role': 'user', 'name': '유리'}, {'content': "아하, 바나나 농담 좋다! 그럼 나도 하나 해볼게.\n\n토끼가 학교에 왜 안 갔을까?\n\n답: 너무 많은 숙제가 있어서 '토끼'를 두고 갔거든! 🐰📚 \n\n어때? 웃겼어?", 'role': 'assistant', 'name': '미성'}, {'content': "유리: 아하, 그거 진짜 재밌다! '토끼'를 두고 갔다는 아이디어가 기발해! 😂\n\n그럼, 우리 농담 대결 한 번 더 해볼까? 내가 하나 해볼게!\n\n왜 수박은 수영을 못 할까?\n\n답: 물에 뜨는 게 힘들어서! 🍉🏊\u200d♂️\n\n너무 웃기지?", 'role': 'user', 'name': '유리'}], summary="유리: 아하, 그거 진짜 재밌다! '토끼'를 두고 갔다는 아이디어가 기발해! 😂\n\n그럼, 우리 농담 대결 한 번 더 해볼까? 내가 하나 해볼게!\n\n왜 수박은 수영을 못 할까?\n\n답: 물에 뜨는 게 힘들어서! 🍉🏊\u200d♂️\n\n너무 웃기지?", cost={'usage_including_cached_inference': {'total_cost': 0.0004896, 'gpt-4o-mini-2024-07-18': {'cost': 0.0004896, 'prompt_tokens': 1080, 'completion_tokens': 546, 'total_tokens': 1626}}, 'usage_excluding_cached_inference': {'total_cost': 0.0004896, 'gpt-4o-mini-2024-07-18': {'

### Using `is_termination_msg`
이번에는 종료 메시지를 "GOOD BYE"로 설정해본다. 이렇게 하면 에이전트가 "GOOD BYE" 라는 메시지를 받았을 때 대화를 자동으로 종료하게 한다.
즉, **특정 키워드**가 포함된 메시지를 **종료 신호**로 인식하게 만들어서,
원하는 조건에서 깔끔하게 대화를 마무리할 수 있게 해주는 방식이다.

In [14]:
miSung = ConversableAgent(
    "미성",
    system_message="Your name is 미성 and you are a part of a duo of comedians.",
    llm_config=llm_config_2,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "good bye" in msg["content"].lower(),
)


In [15]:
result = miSung.initiate_chat(yuRi, message="유리, Good bye라는 단어를 포함한 농담을 해줘.")

미성 (to 유리):

유리, Good bye라는 단어를 포함한 농담을 해줘.

--------------------------------------------------------------------------------
유리 (to 미성):

미성, 알겠어! 여기 하나 해볼게.

"어제 친구에게 작별 인사를 하면서 'Good bye!'라고 했는데, 그는 '왜 이렇게 급해?'라고 하더라. 그래서 내가 말했지, '아니, 나는 그냥 불러서 전화를 끊은 게 아니라, 진짜 Goodbye가 필요한 거야!'"

어때, 마음에 들았어? 😄

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (cfb5c066-941b-4c5b-92b9-046067a47487): Termination message condition on agent '미성' met


## Summary

이번 챕터에서는 **에이전트 간의 대화를 종료하는 여러 가지 방법**을 소개했다. `initiate_chat()`의 파라미터 설정뿐만 아니라, 에이전트 구성에서도 종료 조건을 설정할 수 있다. 하지만 한가지 중요한 점은, **종료 조건이 충족되더라도 대화가 즉시 종료되지 않을 수 있다는 것이다.** 
이는 `ConversableAgent` 클래스의 `human_input_mode` 설정에 따라 달라진다. `NEVER` 모드인 경우, 앞서 설명한 종료 조건이 **즉시 적용 되어 대화를 종료**한다. 그러나 `ALWAYS` 또는 `TERMINATE` 모드에서는 **바로 종료되지 않고 사용자의 개입이나 추가 조건이 필요**할 수 있다. 

이러한 동작 방식과 그 이유는 다음 챕터에서 자세히 다뤄보겠다. 